In [3]:
import os
import numpy as np
import tensorflow as tf
from PIL import Image

# 全局路径配置
OUTPUT_DIR = "C:/data/result/scene_detector"  # 关键帧存放目录
TARGET_FRAMES = 20  # 每个视频的关键帧数
HEIGHT, WIDTH = 224, 224  # 每帧的大小


In [7]:
def load_and_preprocess_frames(frame_paths, target_height, target_width, target_frames):
    """
    从帧路径加载关键帧并预处理为固定大小和数量。
    
    Args:
        frame_paths: 帧文件路径列表（按时间顺序）。
        target_height: 每帧的目标高度。
        target_width: 每帧的目标宽度。
        target_frames: 目标帧数。
    Returns:
        关键帧张量，形状为 (target_frames, height, width, channels)。
    """
    # 加载并调整帧大小
    frames = []
    for frame_path in sorted(frame_paths):  # 确保按帧序排列
        image = Image.open(frame_path)
        image = image.resize((target_width, target_height))
        frames.append(np.array(image) / 255.0)  # 归一化到 [0, 1]
    
    # 确保帧数量一致（补齐或裁剪）
    if len(frames) > target_frames:
        frames = frames[:target_frames]
    elif len(frames) < target_frames:
        padding = target_frames - len(frames)
        frames.extend([np.zeros((target_height, target_width, 3))] * padding)  # 补零帧
    
    return np.stack(frames, axis=0)
def create_dataset_from_frames(output_dir, target_frames, height, width):
    """
    从已存在的关键帧目录构建 TensorFlow 数据集。
    
    Args:
        output_dir: 关键帧存放目录。
        target_frames: 每个视频的关键帧数。
        height: 每帧的高度。
        width: 每帧的宽度。
    Returns:
        TensorFlow 数据集，元素为 (关键帧张量, 标签)。
    """
    frames_list = []
    labels_list = []
    
    # 遍历类别（标签）
    for label in os.listdir(output_dir):
        label_dir = os.path.join(output_dir, label).replace("\\", "/")  # 统一为 /
        if not os.path.isdir(label_dir):
            continue
        
        # 遍历每个视频文件夹
        for video_folder in os.listdir(label_dir):
            video_dir = os.path.join(label_dir, video_folder).replace("\\", "/")  # 统一为 /
            if not os.path.isdir(video_dir):
                continue
            
            # 获取所有帧路径
            frame_paths = [
                os.path.join(video_dir, frame).replace("\\", "/")  # 统一为 /
                for frame in os.listdir(video_dir)
                if frame.endswith((".jpg", ".png"))
            ]
            
            # 跳过空视频文件夹
            if not frame_paths:
                print(f"跳过空视频文件夹: {video_dir}")
                continue
            
            # 加载和预处理帧
            frames = load_and_preprocess_frames(frame_paths, height, width, target_frames)
            frames_list.append(frames)
            labels_list.append(label)
    
    # 将标签转为整数索引
    unique_labels = sorted(set(labels_list))
    label_to_index = {label: idx for idx, label in enumerate(unique_labels)}
    labels_list = [label_to_index[label] for label in labels_list]
    
    # 转为 TensorFlow 数据集
    frames_tensor = tf.convert_to_tensor(np.array(frames_list), dtype=tf.float32)
    labels_tensor = tf.convert_to_tensor(np.array(labels_list), dtype=tf.int32)
    return tf.data.Dataset.from_tensor_slices((frames_tensor, labels_tensor)), label_to_index

In [1]:
import numpy as np
from PIL import Image
from scipy.ndimage import sobel

def load_and_preprocess_frames_custom(frame_paths, target_height, target_width, target_frames):
    """
    加载帧并预处理为灰度 + 水平梯度 + 垂直梯度三通道，统一大小和数量。
    
    Args:
        frame_paths: 帧文件路径列表。
        target_height: 每帧的目标高度。
        target_width: 每帧的目标宽度。
        target_frames: 目标帧数。
        
    Returns:
        三通道关键帧张量，形状为 (target_frames, height, width, 3)。
    """
    frames = []
    for frame_path in sorted(frame_paths):  # 确保按时间顺序加载
        # 加载图片并调整大小
        image = Image.open(frame_path).convert('L')  # 转为灰度图
        image = image.resize((target_width, target_height))
        image_array = np.array(image, dtype=np.float32) / 255.0  # 归一化到 [0, 1]
        
        # 计算梯度通道
        grad_x = sobel(image_array, axis=1)  # 水平方向梯度
        grad_y = sobel(image_array, axis=0)  # 垂直方向梯度
        
        # 归一化梯度到 [0, 1]
        grad_x = (grad_x - grad_x.min()) / (grad_x.max() - grad_x.min() + 1e-6)
        grad_y = (grad_y - grad_y.min()) / (grad_y.max() - grad_y.min() + 1e-6)
        
        # 构造三通道图像
        three_channel_frame = np.stack([image_array, grad_x, grad_y], axis=-1)
        frames.append(three_channel_frame)

    # 确保帧数量一致（补齐或裁剪）
    if len(frames) > target_frames:
        frames = frames[:target_frames]
    elif len(frames) < target_frames:
        padding = target_frames - len(frames)
        padding_frame = np.zeros((target_height, target_width, 3))  # 零填充帧
        frames.extend([padding_frame] * padding)

    return np.stack(frames, axis=0)

def create_custom_dataset(output_dir, target_frames, height, width):
    """
    使用灰度 + 梯度三通道帧构建 TensorFlow 数据集。
    """
    frames_list = []
    labels_list = []

    for label in os.listdir(output_dir):
        label_dir = os.path.join(output_dir, label).replace("\\", "/")
        if not os.path.isdir(label_dir):
            continue

        for video_folder in os.listdir(label_dir):
            video_dir = os.path.join(label_dir, video_folder).replace("\\", "/")
            if not os.path.isdir(video_dir):
                continue

            # 获取帧路径
            frame_paths = [
                os.path.join(video_dir, frame).replace("\\", "/")
                for frame in os.listdir(video_dir)
                if frame.endswith((".jpg", ".png"))
            ]
            if not frame_paths:
                print(f"跳过空视频文件夹: {video_dir}")
                continue

            # 使用新的方法加载和处理帧
            frames = load_and_preprocess_frames_custom(frame_paths, height, width, target_frames)
            frames_list.append(frames)
            labels_list.append(label)

    # 转换标签
    unique_labels = sorted(set(labels_list))
    label_to_index = {label: idx for idx, label in enumerate(unique_labels)}
    labels_list = [label_to_index[label] for label in labels_list]

    # 转为 TensorFlow 数据集
    frames_tensor = tf.convert_to_tensor(np.array(frames_list), dtype=tf.float32)
    labels_tensor = tf.convert_to_tensor(np.array(labels_list), dtype=tf.int32)
    return tf.data.Dataset.from_tensor_slices((frames_tensor, labels_tensor)), label_to_index


In [2]:
# 构建数据集
dataset, label_map = create_dataset_from_frames(OUTPUT_DIR, TARGET_FRAMES, HEIGHT, WIDTH)


NameError: name 'create_dataset_from_frames' is not defined

In [4]:
dataset, label_map = create_custom_dataset(OUTPUT_DIR, TARGET_FRAMES, HEIGHT, WIDTH)

# 查看一个样本
for frames, label in dataset.take(1):
    print(f"帧形状: {frames.shape}, 标签: {label.numpy()}")
    print(f"通道均值: {tf.reduce_mean(frames, axis=[0, 1, 2]).numpy()}")


跳过空视频文件夹: C:/data/result/scene_detector/0-两手托天理三焦（八段锦）/动作0-10-31
帧形状: (20, 224, 224, 3), 标签: 0
通道均值: [0.56452185 0.5130549  0.5523341 ]


In [6]:

# 打印类别映射表
print("类别映射表:", label_map)

# 检查数据集
for frames, label in dataset.take(1):
    print("关键帧张量形状:", frames.shape)  # 应该是 (20, 224, 224, 3)
    print("标签:", label.numpy())


类别映射表: {'0-两手托天理三焦（八段锦）': 0, '1-左右开弓似射雕（八段锦）': 1, '10鹿抵（五禽戏）': 2, '11鹿奔（五禽戏）': 3, '12鸟伸（五禽戏）': 4, '13鸟飞（五禽戏）': 5, '14其他': 6, '2-调理脾胃单臂举（八段锦）': 7, '3-五劳七伤往后瞧（八段锦）': 8, '4-摇头摆尾去心火（八段锦）': 9, '5-两手攀足固肾腰（八段锦）': 10, '6-攒拳怒目增气力（八段锦）': 11, '7背后七颠百病消（八段锦）': 12, '8虎举（五禽戏）': 13, '9虎扑（五禽戏）': 14}
关键帧张量形状: (20, 224, 224)
标签: 0


In [ ]:
import tensorflow as tf

def extract_labels_from_dataset(dataset):
    """
    从 TensorFlow 数据集中提取所有唯一的标签。
    
    Args:
        dataset: tf.data.Dataset 对象，包含 (frames_tensor, label_tensor)。
    Returns:
        标签的集合（去重）。
    """
    labels = set()
    for _, label in dataset:
        labels.add(label.numpy())  # 将标签添加到集合中
    return labels

# 从已构建的数据集中提取标签
all_labels = extract_labels_from_dataset(dataset)
print("所有标签：", all_labels)



In [5]:
import tensorflow as tf
def save_dataset(dataset, save_path):
    """
    使用最新方法保存 TensorFlow 数据集。
    
    Args:
        dataset: tf.data.Dataset 对象。
        save_path: 保存路径。
    """
    dataset.save(save_path)
    print(f"数据集已保存到: {save_path}")

# 保存数据集
save_path = "C:/data/result/saved_dataset_grey"
save_dataset(dataset, save_path)



数据集已保存到: C:/data/result/saved_dataset_grey


In [1]:
import tensorflow as tf
save_path = "C:/data/result/saved_dataset_grey"
def load_dataset(save_path):
    """
    从指定路径加载 TensorFlow 数据集。
    
    Args:
        save_path: 数据集保存路径。
    Returns:
        加载的 tf.data.Dataset 对象。
    """
    dataset = tf.data.experimental.load(save_path)
    print(f"数据集已从 {save_path} 加载")
    return dataset

# 加载数据集
dataset = load_dataset(save_path)


Instructions for updating:
Use `tf.data.Dataset.load(...)` instead.
数据集已从 C:/data/result/saved_dataset_grey 加载


In [2]:
def remove_last_label(dataset):
    """
    从数据集中移除最后一个标签对应的所有数据。
    
    Args:
        dataset: TensorFlow 数据集，元素为 (关键帧张量, 标签)。
    Returns:
        新的数据集，只包含不属于最后一个标签的数据。
    """
    # 找到数据集中最大标签值（即最后一个标签的索引）
    max_label = tf.reduce_max([label for _, label in dataset])

    # 筛选出不属于最后一个标签的数据
    filtered_dataset = dataset.filter(lambda x, y: y != max_label)

    return filtered_dataset

def reassign_dataset_with_filtered(dataset):
    """
    清除原始 dataset 的引用，并将筛选后的 dataset 重新赋值为 dataset。
    
    Args:
        dataset: 原始未筛选的数据集。
    
    Returns:
        筛选后的数据集，赋值为 dataset。
    """
    filtered_dataset = remove_last_label(dataset)
    del dataset  # 清除原始数据集的引用
    return filtered_dataset

# 使用筛选后的数据集
dataset = reassign_dataset_with_filtered(dataset)

# 验证标签分布
def print_filtered_labels_summary(dataset):
    label_counts = {}
    for _, label in dataset:
        label = label.numpy()
        if label not in label_counts:
            label_counts[label] = 0
        label_counts[label] += 1
    print(f"筛选后的标签分布: {label_counts}")

print_filtered_labels_summary(dataset)


筛选后的标签分布: {0: 49, 1: 50, 2: 50, 3: 50, 4: 50, 5: 50, 6: 50, 7: 50, 8: 50, 9: 50, 10: 50, 11: 50, 12: 50, 13: 50}


In [3]:
# 假设 dataset 是已经处理好的 tf.data.Dataset 对象
# 将 dataset 分为训练集和验证集
train_split = 0.90  # 80% 数据用于训练
val_split = 1 - train_split

# 获取总样本数
total_samples = len(list(dataset))
train_size = int(total_samples * train_split)

# 分割数据集
train_ds = dataset.take(train_size)
val_ds = dataset.skip(train_size)

# 配置训练和验证集
train_ds = train_ds.shuffle(buffer_size=1000).batch(8).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.batch(8).prefetch(tf.data.AUTOTUNE)


In [4]:
del dataset
import gc
gc.collect()  # 强制进行垃圾回收

25

In [ ]:
import tqdm
import random
import pathlib
import itertools
import collections

import cv2
import einops
import numpy as np
import remotezip as rz
import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow as tf
from sklearn.model_selection import train_test_split
import keras
from keras import layers
# 启用 XLA
tf.config.optimizer.set_jit(True)


In [17]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import einops
# 启用 XLA
tf.config.optimizer.set_jit(True)

In [6]:
# 定义生成的每一帧的尺寸
HEIGHT = 224  # 高度为224像素
WIDTH = 224   # 宽度为224像素
class Conv2Plus1D(keras.layers.Layer):
    def __init__(self, filters, kernel_size, padding):
        """
        一个卷积层的组合，首先对空间维度进行卷积操作，
        然后对时间维度进行卷积操作。
        """
        super().__init__()
        self.seq = keras.Sequential([  
            # 空间维度分解卷积
            layers.Conv3D(filters=filters,
                          kernel_size=(1, kernel_size[1], kernel_size[2]),  # 只对高度和宽度进行卷积
                          padding=padding),
            # 时间维度分解卷积
            layers.Conv3D(filters=filters, 
                          kernel_size=(kernel_size[0], 1, 1),  # 只对时间步长进行卷积
                          padding=padding)
        ])

    def call(self, x):
        # 执行序列化卷积操作
        return self.seq(x)
class ResidualMain(keras.layers.Layer):
    """
    模型中的残差模块，包含卷积、层归一化和 ReLU 激活函数。
    """
    def __init__(self, filters, kernel_size):
        super().__init__()
        self.seq = keras.Sequential([
            # 第一个卷积层
            Conv2Plus1D(filters=filters,
                        kernel_size=kernel_size,
                        padding='same'),
            # 层归一化
            layers.LayerNormalization(),
            # ReLU 激活函数
            layers.ReLU(),
            # 第二个卷积层
            Conv2Plus1D(filters=filters, 
                        kernel_size=kernel_size,
                        padding='same'),
            # 层归一化
            layers.LayerNormalization()
        ])

    def call(self, x):
        # 执行序列化的操作
        return self.seq(x)
class Project(keras.layers.Layer):
    """
    通过不同大小的过滤器和下采样，对张量的某些维度进行投影处理。
    """
    def __init__(self, units):
        super().__init__()
        self.seq = keras.Sequential([
            # 全连接层（投影操作）
            layers.Dense(units), 
            # 层归一化
            layers.LayerNormalization()
        ])

    def call(self, x):
        # 执行顺序操作
        return self.seq(x)
def add_residual_block(input, filters, kernel_size):
  """
    Add residual blocks to the model. If the last dimensions of the input data
    and filter size does not match, project it such that last dimension matches.
  """
  out = ResidualMain(filters, 
                     kernel_size)(input)

  res = input
  # Using the Keras functional APIs, project the last dimension of the tensor to
  # match the new filter size
  if out.shape[-1] != input.shape[-1]:
    res = Project(out.shape[-1])(res)

  return layers.add([res, out])


class ResizeVideo(keras.layers.Layer):
    def __init__(self, height, width):
        """
        初始化视频尺寸调整层。

        Args:
            height: 调整后的高度。
            width: 调整后的宽度。
        """
        super().__init__()
        self.height = height
        self.width = width
        # 使用 Keras 的 Resizing 层来调整尺寸
        self.resizing_layer = layers.Resizing(self.height, self.width)

    def call(self, video):
        """
        调整视频张量的尺寸。

        Args:
            video: 表示视频的张量，形状为 (batch, time, height, width, channels)。

        Returns:
            调整为新高度和宽度的视频张量。
        """
        # 解析视频的原始形状：b 表示批次大小，t 表示时间步，h 和 w 表示高度和宽度，c 表示通道数
        old_shape = einops.parse_shape(video, 'b t h w c')

        # 将视频重新排列为单张图像的形式，合并批次和时间维度
        images = einops.rearrange(video, 'b t h w c -> (b t) h w c')

        # 调整每一帧的尺寸
        images = self.resizing_layer(images)

        # 将调整后的图像重新排列为视频的形式，分离批次和时间维度
        videos = einops.rearrange(
            images, '(b t) h w c -> b t h w c',
            t=old_shape['t']
        )
        return videos

input_shape = (None, 20, HEIGHT, WIDTH, 3)  # 输入视频的形状，None 表示批次大小不固定
input = layers.Input(shape=(input_shape[1:]))  # 定义输入层，形状为 (时间步数, 高度, 宽度, 通道数)
x = input

# 初始卷积层：执行 2+1D 卷积操作（空间 + 时间分解）
x = Conv2Plus1D(filters=16, kernel_size=(3, 7, 7), padding='same')(x)
x = layers.BatchNormalization()(x)  # 批量归一化层，规范化每批次的特征
x = layers.ReLU()(x)  # 激活函数 ReLU
x = ResizeVideo(HEIGHT//2, WIDTH//4 )(x)  # 调整视频帧的尺寸到 (HEIGHT/2, WIDTH/2)

# Block 1: 添加第一个残差块并调整尺寸
x = add_residual_block(x, 16, (3, 3, 3))  # 添加残差块，过滤器数为 16，卷积核大小为 3x3x3
x = ResizeVideo(HEIGHT // 4, WIDTH // 4)(x)  # 调整尺寸到 (HEIGHT/4, WIDTH/4)

# Block 2: 添加第二个残差块并调整尺寸
x = add_residual_block(x, 32, (3, 3, 3))  # 过滤器数为 32
x = ResizeVideo(HEIGHT // 8, WIDTH // 8)(x)  # 调整尺寸到 (HEIGHT/8, WIDTH/8)

# Block 3: 添加第三个残差块并调整尺寸
x = add_residual_block(x, 64, (3, 3, 3))  # 过滤器数为 64
x = ResizeVideo(HEIGHT // 16, WIDTH // 16)(x)  # 调整尺寸到 (HEIGHT/16, WIDTH/16)

# Block 4: 添加第四个残差块
x = add_residual_block(x, 128, (3, 3, 3))  # 过滤器数为 128

# 全局平均池化和分类
x = layers.GlobalAveragePooling3D()(x)  # 对时间、空间维度进行全局平均池化，生成特征向量
x = layers.Flatten()(x)  # 展平为 1D 向量
x = layers.Dense(15)(x)  # 全连接层输出 10 个分类

# 定义模型
model = keras.Model(input, x)

# 从训练数据集中获取一个批次的数据
frames, label = next(iter(train_ds))

# 通过 build 方法将模型与输入张量关联，用于可视化或调试
model.build(frames)

# 使用 Keras 提供的工具绘制模型结构
keras.utils.plot_model(
    model,               # 目标模型
    expand_nested=True,  # 展开嵌套的层，例如子模块或自定义层
    dpi=60,              # 设置图片分辨率
    show_shapes=True     # 显示每一层输出的形状
)


# 编译模型
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),  # 使用稀疏分类交叉熵作为损失函数
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),              # 优化器为 Adam，学习率设置为 0.0001
    metrics=['accuracy']                                               # 评估指标为准确率
)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [7]:
history = model.fit(
    x=train_ds,            # 使用训练数据集
    epochs=50,             # 训练 50 轮
    validation_data=val_ds # 使用验证数据集
)



Epoch 1/50


75/75 [==============================] - 113s 830ms/step - loss: 2.7243 - accuracy: 0.1219 - val_loss: 5.9379 - val_accuracy: 0.0000e+00
Epoch 2/50
75/75 [==============================] - 98s 1s/step - loss: 2.3973 - accuracy: 0.1686 - val_loss: 6.1220 - val_accuracy: 0.0000e+00
Epoch 3/50
75/75 [==============================] - 62s 680ms/step - loss: 2.3433 - accuracy: 0.1686 - val_loss: 6.6064 - val_accuracy: 0.0000e+00
Epoch 4/50
75/75 [==============================] - 62s 680ms/step - loss: 2.2936 - accuracy: 0.1669 - val_loss: 6.9151 - val_accuracy: 0.0000e+00
Epoch 5/50
75/75 [==============================] - 62s 680ms/step - loss: 2.2467 - accuracy: 0.1970 - val_loss: 7.2825 - val_accuracy: 0.0000e+00
Epoch 6/50
75/75 [==============================] - 62s 684ms/step - loss: 2.2167 - accuracy: 0.1753 - val_loss: 7.2943 - val_accuracy: 0.0000e+00
Epoch 7/50
75/75 [==============================] - 61s 670ms/step - loss: 2.1564 - accuracy: 0.2120 - val_loss: 7.6847 - val_accur

In [7]:
import tensorflow as tf
from tensorflow.keras import layers

class Conv2Plus1D(layers.Layer):
    def __init__(self, filters, kernel_size, padding, **kwargs):
        super().__init__(**kwargs)
        self.filters = filters
        self.kernel_size = kernel_size
        self.padding = padding
        self.seq = tf.keras.Sequential([
            layers.Conv3D(filters=filters, kernel_size=(1, kernel_size[1], kernel_size[2]), padding=padding),
            layers.Conv3D(filters=filters, kernel_size=(kernel_size[0], 1, 1), padding=padding)
        ])

    def call(self, x):
        return self.seq(x)

    def get_config(self):
        # 确保返回所有参数，包括自定义的
        config = super().get_config()
        config.update({
            'filters': self.filters,
            'kernel_size': self.kernel_size,
            'padding': self.padding
        })
        return config

    @classmethod
    def from_config(cls, config):
        # 通过从配置字典中解构来创建类实例
        return cls(**config)

class ResidualMain(layers.Layer):
    def __init__(self, filters, kernel_size, **kwargs):
        super().__init__(**kwargs)
        self.filters = filters
        self.kernel_size = kernel_size
        self.seq = tf.keras.Sequential([
            Conv2Plus1D(filters=filters, kernel_size=kernel_size, padding='same'),
            layers.LayerNormalization(),
            layers.ReLU(),
            Conv2Plus1D(filters=filters, kernel_size=kernel_size, padding='same'),
            layers.LayerNormalization()
        ])

    def call(self, x):
        return self.seq(x)

    def get_config(self):
        config = super().get_config()
        config.update({
            'filters': self.filters,
            'kernel_size': self.kernel_size
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)


class Project(layers.Layer):
    def __init__(self, units, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.seq = tf.keras.Sequential([
            layers.Dense(units),
            layers.LayerNormalization()
        ])

    def call(self, x):
        return self.seq(x)

    def get_config(self):
        config = super().get_config()
        config.update({
            'units': self.units
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)


class ResizeVideo(layers.Layer):
    def __init__(self, height, width, **kwargs):
        super().__init__(**kwargs)
        self.height = height
        self.width = width
        self.resizing_layer = layers.Resizing(self.height, self.width)

    def call(self, video):
        old_shape = einops.parse_shape(video, 'b t h w c')
        images = einops.rearrange(video, 'b t h w c -> (b t) h w c')
        images = self.resizing_layer(images)
        videos = einops.rearrange(images, '(b t) h w c -> b t h w c', t=old_shape['t'])
        return videos

    def get_config(self):
        config = super().get_config()
        config.update({
            'height': self.height,
            'width': self.width
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])



NameError: name 'model' is not defined

In [11]:
# 假设您已有一个训练好的模型 'model'
save_path =r'C:\data\result\model\200grey_model.h5'

# 保存整个模型
try:
   # model.save_weights(save_path)
    model.save(save_path)

    print(f"模型已保存到：{save_path}")
except Exception as e:
    print(f"保存模型时发生错误: {e}")


#print(f"模型已保存到：{save_path}")


模型已保存到：C:\data\result\model\200grey_model.h5


In [15]:
# 使用 Keras 原生格式保存模型
save_path = r'C:\data\result\model\best_model.keras'
model.save(save_path)

print(f"模型已保存到：{save_path}")


NotImplementedError: 
Layer Conv2Plus1D has arguments ['filters', 'kernel_size', 'padding']
in `__init__` and therefore must override `get_config()`.

Example:

class CustomLayer(keras.layers.Layer):
    def __init__(self, arg1, arg2):
        super().__init__()
        self.arg1 = arg1
        self.arg2 = arg2

    def get_config(self):
        config = super().get_config()
        config.update({
            "arg1": self.arg1,
            "arg2": self.arg2,
        })
        return config

In [ ]:
from tensorflow.keras.models import load_model

# 加载模型（如果是 .keras 格式）
model_path = r'C:\data\result\model\best_model.keras'
model = load_model(model_path)

print("模型已成功加载！")


In [8]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import load_model
import einops

# 定义自定义的 Conv2Plus1D 层，修改以接受 trainable 等参数
class Conv2Plus1D(keras.layers.Layer):
    def __init__(self, filters, kernel_size, padding, **kwargs):
        super().__init__(**kwargs)  # 接受所有传递给父类的参数，包括 trainable
        self.seq = keras.Sequential([  
            # 空间维度卷积
            layers.Conv3D(filters=filters,
                          kernel_size=(1, kernel_size[1], kernel_size[2]),
                          padding=padding),
            # 时间维度卷积
            layers.Conv3D(filters=filters, 
                          kernel_size=(kernel_size[0], 1, 1),
                          padding=padding)
        ])

    def call(self, x):
        return self.seq(x)

# 定义残差块的主要模块
class ResidualMain(keras.layers.Layer):
    def __init__(self, filters, kernel_size, **kwargs):
        super().__init__(**kwargs)  # 接受所有传递给父类的参数，包括 trainable
        self.seq = keras.Sequential([
            Conv2Plus1D(filters=filters, kernel_size=kernel_size, padding='same'),
            layers.LayerNormalization(),
            layers.ReLU(),
            Conv2Plus1D(filters=filters, kernel_size=kernel_size, padding='same'),
            layers.LayerNormalization()
        ])

    def call(self, x):
        return self.seq(x)

# 定义用于投影的层
class Project(keras.layers.Layer):
    def __init__(self, units, **kwargs):
        super().__init__(**kwargs)  # 接受所有传递给父类的参数，包括 trainable
        self.seq = keras.Sequential([
            layers.Dense(units),
            layers.LayerNormalization()
        ])

    def call(self, x):
        return self.seq(x)

# 定义 ResizeVideo 层，确保在加载时也能恢复参数
class ResizeVideo(keras.layers.Layer):
    def __init__(self, height, width, **kwargs):
        super().__init__(**kwargs)
        self.height = height
        self.width = width
        self.resizing_layer = layers.Resizing(self.height, self.width)

    def call(self, video):
        old_shape = einops.parse_shape(video, 'b t h w c')
        images = einops.rearrange(video, 'b t h w c -> (b t) h w c')
        images = self.resizing_layer(images)
        videos = einops.rearrange(images, '(b t) h w c -> b t h w c', t=old_shape['t'])
        return videos

    def get_config(self):
        config = super(ResizeVideo, self).get_config()
        config.update({
            'height': self.height,
            'width': self.width
        })
        return config

# 加载模型函数
# 加载模型函数
def load_my_model(model_path):
    # 定义自定义层
    custom_objects = {
        'Conv2Plus1D': Conv2Plus1D,
        'ResizeVideo': ResizeVideo,
        'ResidualMain': ResidualMain,
        'Project': Project  # 确保 Project 也被添加到 custom_objects 中
    }
    
    # 使用 Keras 加载模型
    model = load_model(model_path, custom_objects=custom_objects)
    
    # 编译模型
    model.compile(
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer=keras.optimizers.Adam(learning_rate=0.0001),
        metrics=['accuracy']
    )
    
    return model


In [ ]:


# 调用加载模型的函数，假设您给的路径是 model_path
model_path = r'C:\data\result\model\best_model.h5'  # 替换为您实际的模型文件路径
model = load_my_model(model_path)

# 可选：打印模型摘要
#model.summary()


In [ ]:
r'C:\data\result\model\best_model.
# 查看模型结构
model.summary()

In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.models import load_model
# 2. 编译模型（如果加载后没有编译，需要重新编译）
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00005),  # 初始学习率
    metrics=['accuracy']
)

# 3. 定义 ReduceLROnPlateau 回调来调整学习率
lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',       # 监控验证集的损失
    factor=0.5,               # 每次减小学习率的因子（例如，factor=0.5 表示每次学习率减半）
    patience=3,               # 如果验证损失在 3 轮内没有改善，就减小学习率
    min_lr=1e-6,              # 设置最小学习率
    verbose=1                 # 输出学习率调整的日志
)

# 4. 继续训练模型
history = model.fit(
    x=train_ds,              # 训练数据集
    epochs=20,               # 训练 50 轮，可以根据需要调整
    validation_data=val_ds,  # 验证数据集
    callbacks=[lr_scheduler] # 添加学习率调整回调
)

In [6]:
#新可以保存


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import load_model
import einops
# 定义生成的每一帧的尺寸
HEIGHT = 224  # 高度为224像素
WIDTH = 224   # 宽度为224像素

class Conv2Plus1D(layers.Layer):
    def __init__(self, filters, kernel_size, padding, **kwargs):
        super().__init__(**kwargs)
        self.filters = filters
        self.kernel_size = kernel_size
        self.padding = padding
        self.seq = tf.keras.Sequential([  
            layers.Conv3D(filters=filters,
                          kernel_size=(1, kernel_size[1], kernel_size[2]),  
                          padding=padding),
            layers.Conv3D(filters=filters, 
                          kernel_size=(kernel_size[0], 1, 1),  
                          padding=padding)
        ])

    def call(self, x):
        return self.seq(x)

    def get_config(self):
        config = super().get_config()
        config.update({
            'filters': self.filters,
            'kernel_size': self.kernel_size,
            'padding': self.padding
        })
        return config

class ResidualMain(layers.Layer):
    def __init__(self, filters, kernel_size, **kwargs):
        super().__init__(**kwargs)
        self.filters = filters
        self.kernel_size = kernel_size
        self.seq = tf.keras.Sequential([
            Conv2Plus1D(filters=filters, kernel_size=kernel_size, padding='same'),
            layers.LayerNormalization(),
            layers.ReLU(),
            Conv2Plus1D(filters=filters, kernel_size=kernel_size, padding='same'),
            layers.LayerNormalization()
        ])

    def call(self, x):
        return self.seq(x)

    def get_config(self):
        config = super().get_config()
        config.update({
            'filters': self.filters,
            'kernel_size': self.kernel_size
        })
        return config

class Project(layers.Layer):
    def __init__(self, units, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.seq = tf.keras.Sequential([
            layers.Dense(units),
            layers.LayerNormalization()
        ])

    def call(self, x):
        return self.seq(x)

    def get_config(self):
        config = super().get_config()
        config.update({
            'units': self.units
        })
        return config
def add_residual_block(input, filters, kernel_size):
  """
    Add residual blocks to the model. If the last dimensions of the input data
    and filter size does not match, project it such that last dimension matches.
  """
  out = ResidualMain(filters, 
                     kernel_size)(input)

  res = input
  # Using the Keras functional APIs, project the last dimension of the tensor to
  # match the new filter size
  if out.shape[-1] != input.shape[-1]:
    res = Project(out.shape[-1])(res)

  return layers.add([res, out])

class ResizeVideo(layers.Layer):
    def __init__(self, height, width, **kwargs):
        super().__init__(**kwargs)
        self.height = height
        self.width = width
        self.resizing_layer = layers.Resizing(self.height, self.width)

    def call(self, video):
        old_shape = einops.parse_shape(video, 'b t h w c')
        images = einops.rearrange(video, 'b t h w c -> (b t) h w c')
        images = self.resizing_layer(images)
        videos = einops.rearrange(images, '(b t) h w c -> b t h w c', t=old_shape['t'])
        return videos

    def get_config(self):
        config = super().get_config()
        config.update({
            'height': self.height,
            'width': self.width
        })
        return config

# 保持原有的代码结构不变
input_shape = (None, 20, HEIGHT, WIDTH, 3)  # 输入视频的形状
input = layers.Input(shape=(input_shape[1:]))  # 输入层
x = input

# 初始卷积层
x = Conv2Plus1D(filters=16, kernel_size=(3, 7, 7), padding='same')(x)
x = layers.BatchNormalization()(x)  
x = layers.ReLU()(x)  
x = ResizeVideo(HEIGHT//2, WIDTH//4)(x)  

# Block 1
x = add_residual_block(x, 16, (3, 3, 3))  
x = ResizeVideo(HEIGHT // 4, WIDTH // 4)(x)  

# Block 2
x = add_residual_block(x, 32, (3, 3, 3))  
x = ResizeVideo(HEIGHT // 8, WIDTH // 8)(x)  

# Block 3
x = add_residual_block(x, 64, (3, 3, 3))  
x = ResizeVideo(HEIGHT // 16, WIDTH // 16)(x)  

# Block 4
x = add_residual_block(x, 128, (3, 3, 3))  

# 全局平均池化和分类
x = layers.GlobalAveragePooling3D()(x)  
x = layers.Flatten()(x)  
x = layers.Dense(14)(x)  # 输出类别数为15

# 添加Softmax层输出概率
x = layers.Softmax()(x)  # Softmax层，输出概率

# 定义模型
model = tf.keras.Model(input, x)

# 编译模型
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),  # 设置from_logits=False，因为Softmax已被添加
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    metrics=['accuracy']
)




In [8]:
history = model.fit(
    x=train_ds,            # 使用训练数据集
    epochs=150,             # 训练 50 轮
    validation_data=val_ds # 使用验证数据集
)


Epoch 1/150
79/79 [==============================] - 179s 2s/step - loss: 2.3668 - accuracy: 0.1669 - val_loss: 6.4392 - val_accuracy: 0.0000e+00
Epoch 2/150
79/79 [==============================] - 178s 2s/step - loss: 2.2939 - accuracy: 0.1955 - val_loss: 8.7173 - val_accuracy: 0.0000e+00
Epoch 3/150
79/79 [==============================] - 179s 2s/step - loss: 2.2059 - accuracy: 0.2067 - val_loss: 7.3614 - val_accuracy: 0.0000e+00
Epoch 4/150
79/79 [==============================] - 180s 2s/step - loss: 2.1756 - accuracy: 0.2067 - val_loss: 7.3470 - val_accuracy: 0.0000e+00
Epoch 5/150
79/79 [==============================] - 169s 2s/step - loss: 2.1958 - accuracy: 0.2130 - val_loss: 10.6515 - val_accuracy: 0.0000e+00
Epoch 6/150
79/79 [==============================] - 167s 2s/step - loss: 2.1155 - accuracy: 0.2178 - val_loss: 7.6117 - val_accuracy: 0.1286
Epoch 7/150
79/79 [==============================] - 167s 2s/step - loss: 2.0916 - accuracy: 0.2258 - val_loss: 6.9918 - val_ac

In [9]:
history = model.fit(
    x=train_ds,            # 使用训练数据集
    epochs=25,             # 训练 50 轮
    validation_data=val_ds # 使用验证数据集
)

Epoch 1/25
85/85 [==============================] - 68s 648ms/step - loss: 0.6901 - accuracy: 0.8042 - val_loss: 16.1734 - val_accuracy: 0.0000e+00
Epoch 2/25
85/85 [==============================] - 68s 654ms/step - loss: 0.6871 - accuracy: 0.7982 - val_loss: 8.0698 - val_accuracy: 0.0267
Epoch 3/25
85/85 [==============================] - 68s 658ms/step - loss: 0.6826 - accuracy: 0.7878 - val_loss: 10.2522 - val_accuracy: 0.0000e+00
Epoch 4/25
85/85 [==============================] - 68s 653ms/step - loss: 0.6030 - accuracy: 0.8516 - val_loss: 8.5674 - val_accuracy: 0.0000e+00
Epoch 5/25
85/85 [==============================] - 68s 653ms/step - loss: 0.6713 - accuracy: 0.8145 - val_loss: 9.1816 - val_accuracy: 0.0000e+00
Epoch 6/25
85/85 [==============================] - 68s 651ms/step - loss: 0.6181 - accuracy: 0.8398 - val_loss: 8.7538 - val_accuracy: 0.0000e+00
Epoch 7/25
85/85 [==============================] - 68s 653ms/step - loss: 0.5935 - accuracy: 0.8398 - val_loss: 9.7279 

In [10]:
# 编译模型
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),  # 设置from_logits=False，因为Softmax已被添加
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),
    metrics=['accuracy']
)
history = model.fit(
    x=train_ds,            # 使用训练数据集
    epochs=50,             # 训练 50 轮
    validation_data=val_ds # 使用验证数据集
)

Epoch 1/50
85/85 [==============================] - 80s 687ms/step - loss: 0.2955 - accuracy: 0.9570 - val_loss: 9.2853 - val_accuracy: 0.0533
Epoch 2/50
85/85 [==============================] - 75s 735ms/step - loss: 0.2822 - accuracy: 0.9644 - val_loss: 9.1893 - val_accuracy: 0.1067
Epoch 3/50
85/85 [==============================] - 67s 640ms/step - loss: 0.2782 - accuracy: 0.9688 - val_loss: 9.4427 - val_accuracy: 0.1067
Epoch 4/50
85/85 [==============================] - 68s 646ms/step - loss: 0.2732 - accuracy: 0.9674 - val_loss: 9.4592 - val_accuracy: 0.1200
Epoch 5/50
85/85 [==============================] - 66s 627ms/step - loss: 0.2690 - accuracy: 0.9688 - val_loss: 9.3873 - val_accuracy: 0.0667
Epoch 6/50
85/85 [==============================] - 66s 625ms/step - loss: 0.2630 - accuracy: 0.9748 - val_loss: 9.6784 - val_accuracy: 0.1333
Epoch 7/50
85/85 [==============================] - 65s 622ms/step - loss: 0.2626 - accuracy: 0.9718 - val_loss: 9.7526 - val_accuracy: 0.0800

In [10]:
# 编译模型
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),  # 设置from_logits=False，因为Softmax已被添加
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.000001),
    metrics=['accuracy']
)
history = model.fit(
    x=train_ds,            # 使用训练数据集
    epochs=50,             # 训练 50 轮
    validation_data=val_ds # 使用验证数据集
)

Epoch 1/50
79/79 [==============================] - 180s 2s/step - loss: 0.0458 - accuracy: 0.9984 - val_loss: 9.4671 - val_accuracy: 0.1571
Epoch 2/50
79/79 [==============================] - 169s 2s/step - loss: 0.0373 - accuracy: 0.9968 - val_loss: 9.4303 - val_accuracy: 0.1571
Epoch 3/50
79/79 [==============================] - 168s 2s/step - loss: 0.0322 - accuracy: 1.0000 - val_loss: 9.4527 - val_accuracy: 0.1571
Epoch 4/50
79/79 [==============================] - 168s 2s/step - loss: 0.0343 - accuracy: 0.9984 - val_loss: 9.4281 - val_accuracy: 0.1571
Epoch 5/50
79/79 [==============================] - 168s 2s/step - loss: 0.0319 - accuracy: 0.9984 - val_loss: 9.4597 - val_accuracy: 0.1714
Epoch 6/50
79/79 [==============================] - 168s 2s/step - loss: 0.0407 - accuracy: 0.9984 - val_loss: 9.4364 - val_accuracy: 0.1714
Epoch 7/50
79/79 [==============================] - 168s 2s/step - loss: 0.0322 - accuracy: 1.0000 - val_loss: 9.4254 - val_accuracy: 0.1714
Epoch 8/50
79

In [21]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf

def load_and_preprocess_frames(frame_paths, target_height, target_width, target_frames):
    """
    从帧路径加载关键帧并预处理为固定大小和数量。
    """
    frames = []
    for frame_path in sorted(frame_paths):  # 确保按帧序排列
        image = Image.open(frame_path)
        image = image.resize((target_width, target_height))  # 调整大小
        frames.append(np.array(image) / 255.0)  # 归一化到 [0, 1]
    
    # 确保帧数量一致（补齐或裁剪）
    if len(frames) > target_frames:
        frames = frames[:target_frames]
    elif len(frames) < target_frames:
        padding = target_frames - len(frames)
        frames.extend([np.zeros((target_height, target_width, 3))] * padding)  # 补零帧
    
    return np.stack(frames, axis=0)

def predict_from_folder(model, folder_path, target_height, target_width, target_frames):
    """
    用模型预测某文件夹中20帧视频的标签概率。
    """
    # 获取帧路径
    frame_paths = [
        os.path.join(folder_path, frame).replace("\\", "/")  # 统一为 /
        for frame in os.listdir(folder_path)
        if frame.endswith((".jpg", ".png"))
    ]
    if not frame_paths:
        raise ValueError(f"文件夹 {folder_path} 中没有有效帧文件")
    
    # 预处理帧
    input_frames = load_and_preprocess_frames(frame_paths, target_height, target_width, target_frames)
    input_frames = np.expand_dims(input_frames, axis=0)  # 添加批次维度
    
    # 预测
    probabilities = model.predict(input_frames)
    return probabilities


In [24]:
folder_path = r"c:\data\result\scene_detector\4-摇头摆尾去心火（八段锦）\reference_4"

# 确保以下参数与模型一致
HEIGHT = 224
WIDTH = 224
FRAMES = 20

# 预测
probabilities = predict_from_folder(model, folder_path, HEIGHT, WIDTH, FRAMES)
print("模型预测的各个标签概率分布:", probabilities[0])
print("预测的类别索引:", np.argmax(probabilities[0]))


1/1 [==============================] - 0s 27ms/step
模型预测的各个标签概率分布: [2.8525180e-09 3.6167891e-03 1.3197482e-01 6.6177256e-02 3.4500619e-03
 6.3217543e-05 2.9748273e-03 2.9127113e-08 2.7660965e-08 7.9162389e-01
 2.7017288e-06 1.1617964e-04 1.1569661e-09 2.0664370e-09 1.4175920e-07]
预测的类别索引: 9
